In [1]:
from sklearn.datasets import make_blobs
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold
import os
from tempfile import mkstemp
import subprocess
from IPython.display import Image
from IPython.core.display import display
from sklearn.svm import SVC
from sklearn.tree.export import export_graphviz
from sklearn.model_selection import GridSearchCV

import matplotlib.pyplot
from xgboost.sklearn import XGBClassifier
from sklearn import preprocessing
from sklearn.preprocessing import Imputer
from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
from sklearn.grid_search import GridSearchCV
from sklearn import cross_validation

C:\Users\Mustang8\Anaconda3\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
C:\Users\Mustang8\Anaconda3\lib\site-packages\sklearn\grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [2]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.datasets import make_gaussian_quantiles

Input:データセット読み込み

from sklearn.datasets import load_breast_cancer
cancer = load_breast_cancer()

X = pd.DataFrame(cancer.data)

y=pd.DataFrame(cancer.target)

Dataset = pd.concat([ca_data,ca_targe],axis=1)

# Input

データ作成

In [3]:
z1 = pd.DataFrame([1 ,0],  columns = ['z1'], index = ["x1","x2"])
z2 = pd.DataFrame([-1 ,0], columns = ['z2'], index = ["x1","x2"])
z3 = pd.DataFrame([0 ,1],  columns = ['z3'], index = ["x1","x2"])
z4 = pd.DataFrame([0 ,-1], columns = ['z4'], index = ["x1","x2"])

Z = pd.concat([z1,z2,z3,z4], axis = 1).T

Y = pd.DataFrame([1,1,-1,-1], index = ["z1","z2","z3","z4"], columns = ["target"])

In [4]:
D = pd.concat([Z,Y], axis=1)

基本分類器の作成

In [5]:
def h1(x1,x2):
        if x1 > -0.5:
            target = +1
        else:
            target = -1
        return target

def h2(x1,x2):
        if x1 > -0.5:
            target = -1
        else:
            target = +1

        return target        

def h3(x1,x2):
        if x1 > 0.5:
            target = +1
        else:
            target = -1
        return target  

def h4(x1,x2):
        if x1 > 0.5:
            target = -1
        else:
            target = +1
        return target  

def h5(x1,x2):
        if x2 > -0.5:
            target = +1
        else:
            target = -1
        return target  

def h6(x1,x2):
        if x2 > -0.5:
            target = -1
        else:
            target = +1
        return target  

def h7(x1,x2):
        if x2 > 0.5:
            target = +1
        else:
            target = -1
        return target  

def h8(x1,x2):
        if x2 > 0.5:
            target = -1
        else:
            target = +1
        return target  

In [6]:
classifierH = [h1,h2,h3,h4,h5,h6,h7,h8] #分類器まとめる

In [7]:
T = range(3)  #LOOP回数

# Process:

### process1 重み分布の初期化

In [8]:
N = len(D) #データセットの大きさ 図2.2よりmの値
weightsD = np.zeros(((len(T)+2,N))) #正規化されていない重み分布
weightsD[0] = np.ones(N)/N #初期重み分布

Dt = np.zeros((len(T)+2,N)) #正規化されている重み分布
Dt[0] = weightsD[0]         #用いる分類器のラベル
Z = np.zeros(len(T)+2)      #正規化因子

error_rate = np.zeros(len(T)+1) #誤差率
alpha      =np.zeros(len(T)+1) #重みα

In [9]:
Flag = np.zeros((8,4))               #分類器ごとの正答率
mean = np.zeros(8)                   #平均　分類誤差
count =np.zeros(8) #予測と正解の一致回数

best_Classifierr_list = np.zeros(8) #最良の分類器にフラグを立てる

best_Flag = np.zeros((len(T)+1,len(D))) #最良の分類器の f(x)*ht(x)の値格納
Ht = np.zeros((len(T)+1,len(D))) #

In [10]:
for i in range(8): #分類器の個数
    for num in range(len(D)): #特徴量のスケール
        Flag[i][num] = (D["target"][num]) * classifierH[i](D["x1"][num],D["x2"][num])
        if(Flag[i][num]==1):
            count[i] +=1
    
# meanに分類誤差を格納
for i in range(8):
    mean[i] = (N-count[i])/N

In [11]:
min = 1 #最小値を1と仮定して考えていく

for i in range(8):
    if(abs(mean[i]) < min):
        min = mean[i]
for j in range(8):
    if (mean[j] == min): #minに最小分類誤差が格納されているので、それと一致した場合にbest_Classfierにフラグ立てる
        best_Classifierr_list[j] = np.ones(1)

In [12]:
i = 0
j = 0
while(i < 8):
    if( best_Classifierr_list[i] == 1):
        for num in range((len(D))):
            best_Flag[j][num] = Flag[i][num]
            Ht[j][num] =classifierH[i](D["x1"][num],D["x2"][num])
        j+=1
    i = i + 1

### Process2 ~7

In [16]:
Z[0] = 1

for j in range(4):
    for num in range(4):
        if( best_Flag[j][num] == -1 ):
            error_rate[j] = Dt[j][num]
            alpha[j] =  0.5 * np.log( (1- error_rate[j]) / error_rate[j]) #error_rate
        
        weightsD[j+1][num] = weightsD[j][num] * np.exp( - alpha[j] *  best_Flag[j][num])
        
        Z[j+1] = np.sum(weightsD[j+1])
        
        Dt[j+1] = weightsD[j+1] / Z[j+1]
        

In [20]:
error_rate

array([ 0.25      ,  0.16666667,  0.07294902,  0.01803548])

In [19]:
alpha

array([ 0.54930614,  0.80471896,  1.27112388,  1.99860721])

In [28]:
A =np.zeros((4))
B =np.zeros((4))

for j in range(4):
    for num in range(4):
        A[num] = alpha[j] * Ht[j][num]
        B[num] +=A[num]

H = np.sign(B)

In [29]:
H

array([ 1.,  1., -1., -1.])

In [36]:
LOOP_COUNT =[0,1,2,3]

In [37]:
plt.plot(LOOP_COUNT,error_rate)